In [1]:
from netCDF4 import Dataset
from matplotlib import pyplot as plt # import libraries
import pandas as pd
import xarray as xr
import netCDF4
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import os
import numpy.ma as ma
import psycopg2

In [2]:
#get all nc data
def get_nc():
    file_list=[]
    for file in os.listdir('/home/jed/Downloads/india'):
        if file.endswith(".nc4"):
            file_list.append(file)
            
    return file_list   
print(get_nc())

In [3]:
#load only selected nc data
def load_nc(file_name):
    nc = Dataset(file_name, 'r')
    return nc

In [4]:
#select a dataset based on index,visualize with print
nc_data = get_nc()[1]
#print (nc_data)

In [5]:
#load selected dataset, visualize with print
dataset = load_nc(nc_data)
# print (a)

In [6]:
#load selected variables,
dataset.variables.keys()

odict_keys(['time', 'longitude', 'latitude', 'surface_temperature', 'temperature_at_5m', 'surface_salinity', 'salinity_at_5m', 'drifter_velocity'])

In [15]:
#visualize variable properties
dataset.variables['longitude']  #where longitude is key from the above

<class 'netCDF4._netCDF4.Variable'>
float64 longitude(time)
    _FillValue: 0.0
    long_name: Longitude of AOML Drifter measurement
    valid_max: -92.536
    valid_min: -125.029
    axis: X
    standard_name: longitude
    units: degrees_east
    coverage_content_type: coordinate
unlimited dimensions: 
current shape = (30817,)
filling on

In [18]:
#create list of variables you want to select
variables = ['latitude', 'longitude', 'surface_temperature', 'temperature_at_5m']


#fubction that will siaplay selected variables in a table
def select_var_data(array, time=''):
    df = pd.DataFrame(columns = array)
    for b in array:
        df[b] = dataset.variables[b][:]
        
    df['source_name'] = dataset.title
    times = dataset.variables[time]
    tt = netCDF4.num2date(times[:],times.units)
    df[time] = tt
    return df.head(15)   

#uncomment the below script to visualize dataframe
select_var_data(variables, time='time')

,latitude,longitude,surface_temperature,temperature_at_5m,source_name,time
0,NaN,NaN,NaN,NaN,AOML_Drifter_SPURS2,2016-08-25 00:00:00
1,NaN,NaN,NaN,NaN,AOML_Drifter_SPURS2,2016-08-25 00:30:00.000003
2,NaN,NaN,NaN,NaN,AOML_Drifter_SPURS2,2016-08-25 00:59:59.999997
3,NaN,NaN,NaN,NaN,AOML_Drifter_SPURS2,2016-08-25 01:30:00
4,NaN,NaN,NaN,NaN,AOML_Drifter_SPURS2,2016-08-25 02:00:00.000003
5,10.057000,-125.002000,29.979956,28.899911,AOML_Drifter_SPURS2,2016-08-25 02:29:59.999997
6,10.058000,-125.001000,29.971847,28.904254,AOML_Drifter_SPURS2,2016-08-25 03:00:00
7,10.058973,-125.000027,29.896216,28.908428,AOML_Drifter_SPURS2,2016-08-25 03:30:00.000003
8,10.060480,-125.000000,29.733146,28.905028,AOML_Drifter_SPURS2,2016-08-25 03:59:59.999997
9,10.062000,-125.000000,29.567721,28.901425,AOML_Drifter_SPURS2,2016-08-25 04:30:00


In [12]:
#export dataframe to csv
csv = select_var_data(variables).to_csv('fisrt_try.csv', header=True, index=False, sep=',')

In [13]:
# connect to default database
conn = psycopg2.connect(host="localhost", port = 5432, database="imgtxt", user="postgres", password="postgres1234")

# Create a cursor object
cur = conn.cursor()

In [ ]:
# Insert DataFrame records one by one.
# spurs1 is my postgres table name and the headers i want to insert the data into are as follows.
# the headers should follow the way the dataframe headers have been aligned or else the will be a mix of data.
#if you look carefully I have 4 colums in my dataframe and have selected 4 columns from my postgres table and they follow

for i,row in select_var_data(variables).iterrows():
    sql = "INSERT INTO spurs1 (lat, lon, temp, air_pressure, source_name) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()